In [1]:
import numpy as np
import cv2

# 学習済みファイルの入力
face_cascade_file  = './haarcascades/haarcascade_frontalface_default.xml'
eye_cascade_file  = './haarcascades/haarcascade_eye_tree_eyeglasses.xml'
nose_cascade_file  = './haarcascades/haarcascade_mcs_nose.xml'
mouth_cascade_file  = './haarcascades/haarcascade_mcs_mouth.xml'

face_cascade = cv2.CascadeClassifier(face_cascade_file)
eye_cascade = cv2.CascadeClassifier(eye_cascade_file)
nose_cascade = cv2.CascadeClassifier(nose_cascade_file)
mouth_cascade = cv2.CascadeClassifier(mouth_cascade_file)

In [ ]:
# カメラを開く
cap = cv2.VideoCapture(0)

# 1フレーム目を読み込み、画像の大きさを登録する
end_flag, c_frame = cap.read()
height, width, channels = c_frame.shape

# ウィンドウの準備
cv2.namedWindow('frame')

# メインループ
while end_flag == True:

    # 画像の取得と顔の検出
    img = c_frame
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_list = face_cascade.detectMultiScale(img_gray, minSize=(100, 100))

    # 検出した顔
    for (x, y, w, h) in face_list:
        #print(face_list)
        
        # 顔の枠を描く
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 225), thickness = 3)
        
        # 顔を切り出した画像
        img_face_gray = img_gray[y: y + h, x: x + w]        
        img_face = img[y: y + h, x: x + w]
        
        # パーツ検出
        eye_list = eye_cascade.detectMultiScale(img_face_gray, minSize=(20, 20), minNeighbors=5)
        nose_list = nose_cascade.detectMultiScale(img_face_gray, minSize=(20, 20), minNeighbors=5)
        mouth_list = mouth_cascade.detectMultiScale(img_face_gray, minSize=(20, 20), minNeighbors=5)        
                
        # 画像中の顔の大きさ（width, height）
        height,width,ch = img_face.shape;      
        
        # 口は下半分にあるはず
        if(isinstance(mouth_list,np.ndarray)):
            list_del = list()
            for i,(ex, ey, ew, eh) in enumerate(mouth_list):
                if(ey+(eh/2) < height*2/3):
                    list_del.append(i)
            mouth_list = np.delete(mouth_list,list_del,0)

        # 目は上半分にあるはず
        if(isinstance(eye_list,np.ndarray)):
            list_del = list()
            for i,(ex, ey, ew, eh) in enumerate(eye_list):
                if(ey+(eh/2) > height/2):
                    list_del.append(i)
            eye_list = np.delete(eye_list,list_del,0)

        
        # 鼻は中心近くにあるはず
        if(isinstance(nose_list,np.ndarray)):
            list_del = list()
            for i,(ex, ey, ew, eh) in enumerate(nose_list):
                if(ey+(eh/2) < height/3 or ey > height*2/3):
                    list_del.append(i)
            nose_list = np.delete(nose_list,list_del,0)
        
        # 表示
        for (ex, ey, ew, eh) in eye_list:
            cv2.rectangle(img_face, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), thickness = 2)         
        for (ex, ey, ew, eh) in nose_list:
            cv2.rectangle(img_face, (ex, ey), (ex + ew, ey + eh), (255, 255, 0), thickness = 2)           
        for (ex, ey, ew, eh) in mouth_list:
            cv2.rectangle(img_face, (ex, ey), (ex + ew, ey + eh), (255, 0, 0), thickness = 2)           
        

    # フレーム表示
    cv2.imshow('frame', img)

    # Escキーで終了
    key = cv2.waitKey(10)
    if key == 27:
        break

    # 次のフレーム読み込み
    end_flag, c_frame = cap.read()
    
cv2.destroyAllWindows()
cap.release()